In [1]:
import torch
import pytorch_lightning as pl
from models_vit import DataModule, ViTLightning
from pathlib import Path

pl.seed_everything(0)

Global seed set to 0


0

In [2]:
# input
model_checkpoint = 'google/vit-base-patch16-224-in21k'
path = Path.home() / 'Desktop/dog_cat'

# setup data
dm = DataModule(data_dir=path, model_name=model_checkpoint)
train_ds, val_ds, test_ds = dm.setup()

Resolving data files:   0%|          | 0/25000 [00:00<?, ?it/s]

Using custom data configuration default-27a599091314e833
Found cached dataset imagefolder (C:/Users/Kevin/.cache/huggingface/datasets/imagefolder/default-27a599091314e833/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)
Loading cached split indices for dataset at C:/Users/Kevin/.cache/huggingface/datasets/imagefolder/default-27a599091314e833/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f\cache-aa336e2dcafe124c.arrow and C:/Users/Kevin/.cache/huggingface/datasets/imagefolder/default-27a599091314e833/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f\cache-66d19859bfbef318.arrow
Loading cached split indices for dataset at C:/Users/Kevin/.cache/huggingface/datasets/imagefolder/default-27a599091314e833/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f\cache-4c6679d53bf67667.arrow and C:/Users/Kevin/.cache/huggingface/datasets/imagefolder/default-27a599091314e833/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5a

In [3]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint


# params
path_checkpoints = 'checkpoints'
max_epochs = 5

# callbacks
# saves top-K checkpoints based on "val_loss" metric
val_checkpoint = ModelCheckpoint(
    save_top_k=1,
    monitor="val_loss",
    mode="min",
    dirpath=path_checkpoints,
    filename="sample-{epoch:02d}-{val_loss:.2f}",
)

# saves last-K checkpoints based on "global_step" metric
latest_checkpoint = ModelCheckpoint(
    save_top_k=1,
    monitor="step",
    mode="max",
    dirpath=path_checkpoints,
    every_n_train_steps=500,
    filename="sample-{epoch:02d}-{global_step}",
)

trainer = Trainer(
    default_root_dir=path_checkpoints,
    gpus=1,
    max_epochs=max_epochs,
    callbacks=[val_checkpoint, latest_checkpoint],
)
model = ViTLightning(model_checkpoint, train_ds, val_ds, test_ds, batch_size=32)
trainer.fit(model)

# Test best model on validation and test set
val_result = trainer.test(model, dataloaders=val_ds, verbose=False)
test_result = trainer.test(model, dataloaders=test_ds, verbose=False)
result = {"test": test_result[0]["test_acc"], "val": val_result[0]["test_acc"]}

C:\Users\Kevin\miniconda3\envs\nlp\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:446: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you ex

Sanity Checking: 0it [00:00, ?it/s]

C:\Users\Kevin\miniconda3\envs\nlp\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\Kevin\miniconda3\envs\nlp\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

ValueError: not enough values to unpack (expected 4, got 3)

In [ ]:
# model, results = train_model()
# print("ViT results", results)

In [ ]:
# from pytorch_lightning import Trainer
# from pytorch_lightning.callbacks import EarlyStopping
#
#
# early_stop_callback = EarlyStopping(
#     monitor='val_loss',
#     patience=3,
#     strict=False,
#     verbose=False,
#     mode='min'
# )
#
# model = ViTLightning(model_checkpoint, train_ds, val_ds, test_ds, batch_size=32)
# trainer = Trainer(gpus=1, callbacks=[EarlyStopping(monitor='validation_loss')])
# trainer.fit(model)

In [ ]:
# torch.save(model.state_dict(), "model/vit_finetune.pt.pt")